Task is to make question answering system for video. Proposed solution is to cut video in 30s-60s audio slices with sliding window, get texts from this slices via ASR model. Then having a qustion get top N related texts and get the answer using QA LLM T5, using as context most related texts

In [2]:
!pip install sentencepiece
!pip install transformers
!pip install pydub
!pip install -U sentence-transformers
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 15.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=e37505ab0a0ed4972d7649fc80c2b6750bdb650b13ec1faec

In [35]:
from transformers import pipeline, T5Tokenizer, T5ForConditionalGeneration, AutoModelWithLMHead, AutoTokenizer
from pydub import AudioSegment
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch
import sentencepiece
import whisper

In [64]:
# Whisper is the best here because it's don't require fine-tuning and its already good enought for transcription.
pipe = pipeline(model="openai/whisper-base")
#pipe = pipeline(model="facebook/wav2vec2-base-960h")
#whisper = whisper.load_model("base")

Downloading:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/290M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/827 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/494k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [84]:
# Best window (based on experiments) is 30 sec, to do bigger windows I need to use whisper library for inference, because transformer don't allow big files

long_audio = AudioSegment.from_mp3('../../NounsAI.mp3')
window_len = 30 * 1000 # 30 sec
window_stride = 10 * 1000 # 10 sec context

windows_texts = []
print(long_audio.duration_seconds, 'long audio seconds dur')
print(int(long_audio.duration_seconds*1000 // (window_len - window_stride)), 'windows number')
for i in range(int(long_audio.duration_seconds*1000 // (window_len - window_stride))):
  curr_window = long_audio[max(0, i*(window_len-window_stride)) : min(long_audio.duration_seconds*1000, i*(window_len-window_stride) + window_len)]
  curr_window.export('curr_window.mp3', format='mp3') # not scalable solution :D
  window_text = pipe('curr_window.mp3', max_new_token=1000)
  #window_text = whisper.transcribe('curr_window.mp3')
  windows_texts.append(window_text)

3080.908820861678 long audio seconds dur
154 windows number


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new

In [7]:
# Best pretrained model based on metrics on sentence semantic search problem
model_sentence_transformer = SentenceTransformer('all-mpnet-base-v2')

def get_embeds(sentence_list: str):
  return model_sentence_transformer.encode(sentence_list)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [85]:
for i in range(len(windows_texts)):
  windows_texts[i]['embedding'] = get_embeds(windows_texts[i]['text'])

In [86]:
windows_embeds = [window_dict['embedding'] for window_dict in windows_texts]

In [39]:
!git clone https://github.com/patil-suraj/question_generation.git
%cd question_generation
from pipelines import pipeline as pipeline_qa

Cloning into 'question_generation'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 299.04 KiB | 1.17 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/content/question_generation/question_generation


In [101]:
# T5 is very good pretrained LLM that dont need fine-tuning - exactly what I need here. Have limitaions in context length :(
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelWithLMHead.from_pretrained("t5-base")
#qa = pipeline_qa("multitask-qa-qg", model="valhalla/t5-base-qa-qg-hl")

def get_answer(question: str, top_k=5):
  question_embed = get_embeds(question)
  cos_scores = util.cos_sim(question_embed, windows_embeds)[0]
  top_results = torch.topk(cos_scores, k=top_k)

  answers = []
  for i in range(top_k):
    top_text = windows_texts[top_results[1][i]]['text']
    input_ids = tokenizer(f'question: {question} context: {top_text} </s>', return_tensors="pt")
    outputs = model.generate(input_ids=input_ids['input_ids'], attention_mask=input_ids['attention_mask'])
    answers.append((tokenizer.decode(outputs[0], skip_special_tokens=True), top_text))

  concated_context = ' '.join([windows_texts[top_results[1][i]]['text'] for i in range(top_k)]) + ' </s>'
  input_ids = tokenizer(f'question: {question} context: {concated_context}', return_tensors="pt").input_ids
  outputs = model.generate(input_ids)
  answers.append((tokenizer.decode(outputs[0], skip_special_tokens=True), concated_context))

  return answers

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [103]:
get_answer('How often is a group doing summaries of The Noun Square?')

[('every single day',
  " It's not my time to. Awesome. Thanks for the thanks. I mean, thanks for the shout because that helps us, you know, know that we're building the right direction. So definitely will let you know as soon as that's up and get your feedback on how we can improve it. Yeah. And in the meantime, there was a group doing summaries of Nounsquares every single day. They were funded in our recent prop house. So that's definitely a good solution in the interim. Oh, cool. I'll send you a link."),
 ('human',
  " course that's human beings doing it. So I'm sure even they would even they would be happy to have some AI generated summaries to make their job easier. I'm sure. Yeah, I mean, so that's the vision, right? There's AI generated summary and then someone who was one who went and saw the now, you know, now the clock or the now square or whatever gets uploaded. They can go in and they know if the edit is correct or not, right? Like there will be edits that are missing infor

In [104]:
get_answer('What general questions were asked in this meeting?')

[('make your own judgment, and if you want to create another edit that will then use to',
  " the AI summarized content to the raw content that we've got passed in. So that might not be like the raw NP3 file, maybe that's the transcription of the whole meeting. But we always want you to be able to drill down to the full proposal, make your own judgment, and if you want to create another edit that will then use to improve the edits made by Rocco in the future. So yeah, that was a lot of me, Randland. Hopefully you guys are not tired of me talking at you for 30 minutes."),
 ('what kind of data is really easy?',
  " the knowledge base. Let me try to pull up the, my notes for the notion on the second part of the knowledge base, which I started thinking about. Yeah, I don't have too much to share there just yet. Just starting to think about what kind of questions can we really nail for people? Like, what kind of data is really easy? Is what kind of questions are often asked and really easy 

In [105]:
get_answer('What were the important points of this meeting')

[("the AI summarized content to the raw content that we've got passed in",
  " the AI summarized content to the raw content that we've got passed in. So that might not be like the raw NP3 file, maybe that's the transcription of the whole meeting. But we always want you to be able to drill down to the full proposal, make your own judgment, and if you want to create another edit that will then use to improve the edits made by Rocco in the future. So yeah, that was a lot of me, Randland. Hopefully you guys are not tired of me talking at you for 30 minutes."),
 ("the Nance Square or Nanna Clock, they'd be able to just upload",
  " of scroll through the non-oplock. But if we have like a text summary of the conversation, I think that it would be pretty cool. That's the goal. We should have that by next week. We should have the ability for any button to upload that recording. We'll probably, we'll probably wait less people at first. So the Nance Square or Nanna Clock, they'd be able to just u

In [106]:
get_answer('What projects are the AI Pod working on?')

[('paper implement or a proof of concept',
  " Use case for machine learning that they think that they're not sure if it could be useful, but potentially could be really useful. I do have a lot of friends who do AIML research that are up for doing like little projects that like AI pod could fund them for a little bit of an exploration to try and get like a paper implement or a proof of concept working. So we can kind of like, you know, there's a little bit of a small grant forum to kind of prove something out."),
 ('AI generated art',
  " I'm also gonna be able to create it. So, we'll be able to, but I think you guys, you guys, you guys, you're gonna be able to create so many more things that we're never, we can't even think about. Any questions about kind of like this side of things. So this is one of our main pillars that we talked a lot about. AI generated art. Really excited with how this got, you know, huge shout out to Hawk for really putting a lot of work in it, get a working de

In [107]:
get_answer('What is the name of the project?')

[('AI pod',
  " Use case for machine learning that they think that they're not sure if it could be useful, but potentially could be really useful. I do have a lot of friends who do AIML research that are up for doing like little projects that like AI pod could fund them for a little bit of an exploration to try and get like a paper implement or a proof of concept working. So we can kind of like, you know, there's a little bit of a small grant forum to kind of prove something out."),
 ("'The 'T'",
  " And then from there, that's cool. We either see it work or we see it don't. And then we've funded that a little bit of exploration and then it can continue to grow. So I guess something to agree the answer question, it'd be something that we've seen somebody else do or seen someone else do a part of that we think would be specifically useful for us and that we don't see other tools available. And something that might be useful for the broader community as well"),
 ('proposals',
  " demos f

Due to large duration of origin video I was needed to cut it to small pieces. Long context and semantc search works not easily with public pretrained model, that's why here is also limitations for duration of context.
Due to cutting original video we have some performance decreasing coz models lose context. I tried to solve this by using sliding window, but best approach is to use model for Large context ASR.

One of possible improvements is to using rerank model to rerank answers, because we have top N answers and we wat to choose best from this top N